In [1]:
import sys, os, re; sys.path = (["../src/", "../"] if re.match(r'^(\w\:\\)|(/)', os.getcwd()) else [])+ sys.path 

import qube
%qubed

%load_ext autoreload
%autoreload 2
from qube import lookup
from qube.core.basics import Instrument, FuturesInfo
from qube.utils.marketdata.binance import get_binance_symbol_info_for_type
from qube.utils.pandas import scols

 >  [dev] installed cython rebuilding hook


2024-02-08 13:42:42.230 | INFO     | qube.core.lookups:_update_binance:127 - Loaded 276 for binance.um
2024-02-08 13:42:42.294 | INFO     | qube.core.lookups:_update_binance:127 - Loaded 62 for binance.cm
2024-02-08 13:42:42.310 | INFO     | qube.core.lookups:_update_binance:127 - Loaded 2473 for binance
2024-02-08 13:42:42.440 | INFO     | qube.core.lookups:_update_dukas:91 - Updates 7 for DUKASCOPY



                   ╻
   ┏┓      ╻       ┃  ┏┓       Quantitative Backtesting Environment 
   ┃┃  ┓┏  ┣┓  ┏┓  ┃  ┏┛       
   ┗┻  ┗┻  ┗┛  ┗   ┃  ┗━       (c) 2024,  ver. Dev
                   ╹       



# mstruct -> Struct

In [14]:
from qube.utils import Struct

In [15]:
y = Struct(Struct(x=1), b=Struct(x=1), c = 555)
y

Struct(x=1, b=Struct(x=1), c=555)

In [16]:
y | {'a1': 2}

Struct(x=1, b=Struct(x=1), c=555, a1=2)

In [37]:
y.ast = 1

In [111]:
dict(s=1)

{'s': 1}

In [38]:
y

Struct(a=2, b=Struct(x=1), ast=1)

# Market info and symbols

In [36]:
s0 = lookup['BINANCE:ETH.*']
s1 = lookup['DUKAS:EURGBP']

In [ ]:
(
    lookup.find_aux_instrument_for(s0[0], 'USDT'), 
    lookup.find_aux_instrument_for(s0[1], 'USDT'), 
    lookup.find_aux_instrument_for(s1[0], 'USD'),
)